In [7]:
import torch
from torchvision.transforms import transforms
from model import Yolov1
from loss import YoloLoss
import torch.optim as optim
from dataset import VOCDataset
from tqdm import tqdm
from torch.utils.data import DataLoader
from utils import (
    intersection_over_union,
    non_max_suppression,
    mean_average_precision,
    get_bboxes
    )

In [8]:
#Hyperparameters etc.
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
LEARNING_RATE = 1e-2
WEIGHT_DECAY = 0.0
LOAD_MODEL = False
IMG_DIR = "data/images"
LABEL_DIR = "data/labels"
BATCH_SIZE = 16
PIN_MEMORY = True
EPOCH = 10


In [9]:
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms
    def __call__(self, img, bboxes):
        for i in self.transforms:
            img, bbox = i(img), bboxes
            
        return img, bbox
    
transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor()])

In [10]:
def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    
    train_loss = []
    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)
        
        y_train = model(x)
        loss = loss_fn(y_train, y)
        train_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
#         update progress
        loop.set_postfix(loss=loss.item())
        
    print(f"train_loss: {sum(train_loss)/len(train_loss)}")
        

In [11]:
def main():
    model = Yolov1(split_size=7, num_boxes= 2, num_classes= 20).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    loss_fn = YoloLoss()
    
    if LOAD_MODEL:
        pass
    
    
    train_dataset = VOCDataset(
        "data/8examples.csv",
        IMG_DIR,
        LABEL_DIR,
        transform=transform
    )
    
    test_dataset = VOCDataset(
        "data/test.csv",
        IMG_DIR,
        LABEL_DIR,
        transform=transform
    )
    
    train_dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        pin_memory=PIN_MEMORY,
        shuffle = True,
        drop_last=False       
    
    )
        
    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        pin_memory=PIN_MEMORY,
        shuffle = True,
        drop_last=True       
    
    )
    
    for epoch in range(EPOCH):
        pred_bboxes, target_bboxes = get_bboxes(train_dataloader, model, iou_threshold=0.5, threshold=0.4)
        
        mean_avg_prec = mean_average_precision(pred_bboxes, target_bboxes)
           
        train_fn(train_dataloader, model, optimizer, loss_fn)
        print(f"train mAP for epoch {EPOCH}: {mean_avg_prec}")
            
    

In [ ]:
ipynb

In [12]:
if __name__ == "__main__":
    main()

train mAP for epoch 10: 0.0


100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.85it/s, loss=387]


train_loss: 387.0162658691406
train mAP for epoch 10: 0.0


100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.96it/s, loss=7.15e+5]


train_loss: 714837.375
train mAP for epoch 10: 0.0


...

train_loss: 21302.40234375
train mAP for epoch 10: 0.0


100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.97it/s, loss=2.36e+4]


train_loss: 23589.96484375
train mAP for epoch 10: 0.0


100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.92it/s, loss=4.47e+4]

train_loss: 44687.234375
